In [ ]:
# This cell installs HOOMD in Google Colab. Delete it if you run locally
# !pip install -q condacolab
# import condacolab

# condacolab.install_from_url(
#     'https://github.com/glotzerlab/hoomd-workshop/releases/download/2022.0.0/hoomd-workshop-2022.0-Linux-x86_64.sh'
# )

In [ ]:
from render_octahedra import render

# Introducing HOOMD-blue

[HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/)

## Core objects

HOOMD-blue is an object-oriented Python package. First, import the package:

In [ ]:
import hoomd

### Selecting a device

Choose the hardware device to use:

In [ ]:
# gpu = hoomd.device.GPU()

Try creating a CPU device:

In [ ]:
# Complete the code
# cpu =
cpu = hoomd.device.CPU()

### Creating a Simulation

Now, you can instantiate a **Simulation** with the chosen device.

In [ ]:
sim = hoomd.Simulation(device=cpu, seed=1)

A newly constructed **Simulation** has no **state**:

In [ ]:
print(sim.state)

### Populating the simulation state

Placing particles in the initial configuration snapshot:

In [ ]:
import freud

In [ ]:
snapshot = hoomd.Snapshot()
box, positions = freud.data.UnitCell.fcc().generate_system(num_replicas=3,
                                                           scale=1.5)
snapshot.particles.N = len(positions)
snapshot.particles.position[:] = positions[:]
snapshot.particles.types = ['octahedron']
snapshot.configuration.box = [box.Lx, box.Ly, box.Lz, 0, 0, 0]
render(snapshot)

Initialize the simulation **state** from the snapshot:

In [ ]:
sim.create_state_from_snapshot(snapshot)

You can also create a simulation state from a GSD file. Try initializing a new simulation from the extant file `octahedra.gsd`:

In [ ]:
# Complete the 2 lines of code
# sim2 = hoomd.Simulation(
# sim2.

sim2 = hoomd.Simulation(device=cpu)
sim2.create_state_from_gsd('octahedra.gsd')

### Adding the integrator and other operations

The **integrator** determines what kind of simulation HOOMD-blue executes. Here, we will perform hard particle Monte Carlo simulations of octahedra.

The **ConvexPolyhedron** **integrator** implements HPMC simulations - Create one:

In [ ]:
mc = hoomd.hpmc.integrate.ConvexPolyhedron()

Set the `shape` *property* to define the **particle shape**.
A convex polyhedron is defined by the convex hull of a [set of vertices](https://en.wikipedia.org/wiki/Octahedron):

In [ ]:
mc.shape['octahedron'] = dict(vertices=[
    (-0.5, 0, 0),
    (0.5, 0, 0),
    (0, -0.5, 0),
    (0, 0.5, 0),
    (0, 0, -0.5),
    (0, 0, 0.5),
])

Set the maximum trial displacement `d` and rotation `a`:

In [ ]:
mc.d['octahedron'] = 0.15
mc.a['octahedron'] = 0.2

Add the HPMC **integrator** to the **Simulation** operations:

In [ ]:
sim.operations += mc

## Running the simulation

The GSD writer will save simulation snapshots to a trajectory file:

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(1000))

Add `gsd_writer` to the simulation's operations:

In [ ]:
# Add code here

sim.operations += gsd_writer

Run the simulation for 10,000 steps:

In [ ]:
# Add code here
sim.run(10000)

In [ ]:
import gsd.hoomd

with gsd.hoomd.open('trajectory.gsd') as trajectory:
    image = render(trajectory[-1])
image

Let's check the translation trial move acceptance:

In [ ]:
accepted_moves, rejected_moves = mc.translate_moves
accepted_moves / (accepted_moves + rejected_moves)

What is the rotation trial move acceptance?

In [ ]:
# Add code here

Add a **Tuner** that adjusts `a` and `d` to achieve a target acceptance ratio of 0.2:

In [ ]:
move_size_trigger = hoomd.trigger.Periodic(10)
tune_move_size = hoomd.hpmc.tune.MoveSize.scale_solver(
    moves=['a', 'd'],
    target=0.2,
    trigger=move_size_trigger,
    max_translation_move=1.0,
    max_rotation_move=0.5)
sim.operations += tune_move_size

Run the simulation for 1000 steps:

In [ ]:
# Add code here
sim.run(1000)

In [ ]:
accepted_moves, rejected_moves = mc.translate_moves
accepted_moves / (accepted_moves + rejected_moves)

In [ ]:
mc.d['octahedron']

## Exercise: Compressing the system to a target packing fraction

Find an operation in the [HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/) that you can use to compress the system.